<a href="https://colab.research.google.com/github/Srishti0811/AI-Driven-Mental-Disorder-Detection/blob/main/Sibbir_Data_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibbir","key":"c9b1f65daec00b23325e5e866738f3b6"}'}

In [2]:
!pip install kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d thienkhonghoc/affectnet

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown
 99% 1.73G/1.75G [00:16<00:00, 110MB/s] 
100% 1.75G/1.75G [00:16<00:00, 114MB/s]


In [5]:
!unzip /content/affectnet.zip -d /content/affectnet

Streaming output truncated to the last 5000 lines.
  inflating: /content/affectnet/AffectNet/train/6/image0016336.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016340.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016343.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016344.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016347.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016354.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016358.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016363.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016364.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016367.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016369.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016370.jpg  
  inflating: /content/affectnet/AffectNet/train/6/image0016381.jpg  
  inflating: /content/affectnet/AffectNet/train/6/im

In [6]:
import os

dataset_path = '/content/affectnet'
print("Files in the dataset folder:")
print(os.listdir(dataset_path))

Files in the dataset folder:
['AffectNet']


In [7]:
affectnet_path = '/content/affectnet/AffectNet'
print("Files in the AffectNet folder:")
print(os.listdir(affectnet_path))

Files in the AffectNet folder:
['train', 'test', 'val']


In [8]:
train_path = os.path.join(affectnet_path, 'train')
val_path = os.path.join(affectnet_path, 'val')
test_path = os.path.join(affectnet_path, 'test')

print("Files in the train folder:")
print(os.listdir(train_path))
print("\nFiles in the val folder:")
print(os.listdir(val_path))
print("\nFiles in the test folder:")
print(os.listdir(test_path))

Files in the train folder:
['5', '1', '0', '2', '6', '3', '4', '7']

Files in the val folder:
['5', '1', '0', '2', '6', '3', '4', '7']

Files in the test folder:
['5', '1', '0', '2', '6', '3', '4', '7']


In [9]:
import os

# Define the base path (where the AffectNet folder is located)
base_path = '/content/affectnet/AffectNet'

# Define paths for train, val, and test folders
train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test')

print(f"Train path: {train_path}")
print(f"Validation path: {val_path}")
print(f"Test path: {test_path}")

Train path: /content/affectnet/AffectNet/train
Validation path: /content/affectnet/AffectNet/val
Test path: /content/affectnet/AffectNet/test


In [10]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 15.2 MB/s eta 0:00:00


In [11]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight

# Emotion map for AffectNet labels
emotion_map = {
    0: "Neutral",
    1: "Happy",
    2: "Sad",
    3: "Surprise",
    4: "Fear",
    5: "Disgust",
    6: "Anger",
    7: "Contempt"
}

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection

def crop_face(image):
    """Detect and crop the face region using Mediapipe; return original image if no face is detected."""
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(image)
        if results.detections:
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = image.shape
            x_min = int(bbox.xmin * w)
            y_min = int(bbox.ymin * h)
            x_max = min(x_min + int(bbox.width * w), w)
            y_max = min(y_min + int(bbox.height * h), h)
            if x_max - x_min > 0 and y_max - y_min > 0:
                return image[y_min:y_max, x_min:x_max]
    return image

def load_images_and_labels(base_path, emotion_map, max_images_per_class=300, target_size=(380,380)):
    images = []
    labels = []
    for emotion_id in emotion_map.keys():
        folder = os.path.join(base_path, str(emotion_id))
        count = 0
        for fname in os.listdir(folder):
            if count >= max_images_per_class:
                break
            path = os.path.join(folder, fname)
            img = cv2.imread(path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = crop_face(img)
                if img is None or img.size == 0:
                    continue
                try:
                    img = cv2.resize(img, target_size)
                except Exception as e:
                    print(f"Error resizing image {path}: {e}")
                    continue
                images.append(img)
                labels.append(emotion_id)
                count += 1
    return np.array(images), np.array(labels)

base_path = '/content/affectnet/AffectNet'
train_path = os.path.join(base_path, 'train')
val_path   = os.path.join(base_path, 'val')
test_path  = os.path.join(base_path, 'test')

print("Loading training data...")
train_images, train_labels = load_images_and_labels(train_path, emotion_map, max_images_per_class=300)
print(f"Training data: {len(train_images)} images")

print("Loading validation data...")
val_images, val_labels = load_images_and_labels(val_path, emotion_map, max_images_per_class=300)
print(f"Validation data: {len(val_images)} images")

print("Loading test data...")
test_images, test_labels = load_images_and_labels(test_path, emotion_map, max_images_per_class=300)
print(f"Testing data: {len(test_images)} images")

# Preprocess images using EfficientNet's preprocess_input
print("Preprocessing images...")
train_images = preprocess_input(train_images)
val_images   = preprocess_input(val_images)
test_images  = preprocess_input(test_images)

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes=8)
val_labels   = to_categorical(val_labels, num_classes=8)
test_labels  = to_categorical(test_labels, num_classes=8)

# # Compute class weights to handle class imbalance
# class_weights = class_weight.compute_class_weight('balanced', np.unique(np.argmax(train_labels, axis=1)), np.argmax(train_labels, axis=1))
# class_weights = dict(enumerate(class_weights))

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
train_generator = datagen.flow(train_images, train_labels, batch_size=32)

# Build the emotion classifier model using EfficientNetB4
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380,380,3))
base_model.trainable = False  # Freeze base model initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

print("Starting training with EfficientNetB4...")
model.fit(train_generator, epochs=40, validation_data=(val_images, val_labels),
          callbacks=[early_stopping, reduce_lr, checkpoint], )

# Gradual fine-tuning: Unfreeze the lower half of the base model
base_model.trainable = True
for layer in base_model.layers[:len(base_model.layers)//2]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
print("Starting fine-tuning...")
model.fit(train_generator, epochs=30, validation_data=(val_images, val_labels),
          callbacks=[early_stopping, reduce_lr, checkpoint], )

model.save('emotion_classifier_EfficientNetB4.keras')
print("Emotion classifier model saved as 'emotion_classifier_EfficientNetB4.keras'")

Loading training data...
Training data: 2400 images
Loading validation data...
Validation data: 788 images
Loading test data...
Testing data: 2400 images
Preprocessing images...
71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Starting training with EfficientNetB4...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1376 - loss: 7.3742
Epoch 1: val_loss improved from inf to 6.45181, saving model to best_model.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.1378 - loss: 7.3733 - val_accuracy: 0.1662 - val_loss: 6.4518 - learning_rate: 1.0000e-04
Epoch 2/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1938 - loss: 7.0522
Epoch 2: val_loss improved from 6.45181 to 6.28919, saving model to best_model.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.1937 - loss: 7.0513 - val_accuracy: 0.2424 - val_loss: 6.2892 - learning_rate: 1.0000e-04
Epoch 3/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2213 - loss: 6.7793
Epoch 3: val_loss improved from 6.28919 to 6.19095, saving model to best_model.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.2213 - loss: 6.7786 - val_accuracy: 0.2462 - val_loss: 6.1910 - learning_rate: 1.0000e-04
Epoch 4/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2554 